In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
from torchvision.transforms import transforms
import torch.optim as optim
from torchinfo import summary
import torchvision


device = torch.device("cuda" if torch.cuda.is_available  else "cpu")
print("Using device: ", device)


In [ ]:
model = torchvision.models.resnet18(weights = 'DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 10)

# summary(model, (1, 3, 224, 224), device = "cpu")
model.to(device) # move to gpu
print("--- Model Loaded --- ")

### Simple DataParallel to use Kaggle T4 2 GPus

In [ ]:
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = nn.DataParallel(model)

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_set = torchvision.datasets.CIFAR10(root = "./data", download = True, train = True, transform = transform)

train_loader = DataLoader(train_set, batch_size = 32, shuffle = True, num_workers = 2)

In [ ]:
num_epochs = 5


for i, (images, labels) in enumerate(trainloader, start = 0):
    running_loss = 0.0
    images, labels = images.to(device), labels.to(device)

    optimizer.zero_grad(set_to_none = True)
    prediction = model(images)
    loss = criterion(prediction, labels)

    pred_idx = prediction.argmax(dim = 1)
    loss.backward()
    optimizer.step()

    running_loss = loss.item() 


    if i % 100 == 99: # print every 100 mini-batches
       print(f"Epoch : {i + 1} ; Loss : {running_loss  / 100:.3f}")
       running_loss = 0.0

print("Finished Training")




### Distributed DataParallelism - (need fix)

In [ ]:
%%writefile train.py

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import os
from tqdm import tqdm  # Import tqdm

# Initialize the process group for DDP (for multi-GPU training)
def init_process_group(rank, world_size):
    os.environ['RANK'] = str(rank)
    os.environ['WORLD_SIZE'] = str(world_size)
    os.environ['MASTER_ADDR'] = 'localhost'  # Set master address
    os.environ['MASTER_PORT'] = '12355'  # Set master port (use an unused port)
    
    dist.init_process_group(backend='nccl', init_method='env://', rank=rank, world_size=world_size)

# Set device to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Model definition (ResNet18 example)
model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 10)  # Modify for 10 output classes (CIFAR10)
model = model.to(device)

# Manually set rank (0 for first GPU, 1 for second GPU)
rank = torch.cuda.current_device()  # This sets the rank based on the current device
world_size = torch.cuda.device_count()  # Number of GPUs available (2 in Kaggle)

# Initialize the process group and wrap the model with DDP
init_process_group(rank, world_size)

# DDP requires wrapping the model after initializing the process group
model = DDP(model, device_ids=[rank])

# Dataset and DataLoader
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Use tqdm for a progress bar in the loop
    loop = tqdm(enumerate(trainloader, 0), total=len(trainloader), desc=f"Epoch {epoch+1}/{num_epochs}", dynamic_ncols=True)

    for i, data in loop:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize the model
        optimizer.step()

        # Update the progress bar with the loss
        running_loss += loss.item()
        if i % 100 == 99:  # Print every 100 mini-batches
            loop.set_postfix(loss=running_loss / 100)
            running_loss = 0.0  # Reset running loss after logging

print("Finished Training")


In [ ]:
!python train.py

# Reference: https://github.com/dnddnjs/pytorch-multigpu/blob/master/dist_parallel/train.py

In [ ]:
!pip install tensorboardX -q

In [1]:
%%writefile train.py

import os
import time
import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import torch.distributed as dist
import torch.multiprocessing as mp
import torch.utils.data.distributed

# from model import pyramidnet
import argparse
from tensorboardX import SummaryWriter


parser = argparse.ArgumentParser(description='cifar10 classification models')
parser.add_argument('--lr', default=0.1, help='')
parser.add_argument('--resume', default=None, help='')
parser.add_argument('--batch_size', type=int, default=768, help='')
parser.add_argument('--num_workers', type=int, default=4, help='')
parser.add_argument("--gpu_devices", type=int, nargs='+', default=None, help="") 
# nargs can take more than value, as a list  --> if only one value is passed it is treated as non-iter then it may throw error TypeError: 'list' object cannot be interpreted as an integer

parser.add_argument("--epochs", type=int, default=None, help="")

parser.add_argument('--gpu', default=None, type=int, help='GPU id to use.')
parser.add_argument('--dist-url', default='tcp://127.0.0.1:3456', type=str, help='')
parser.add_argument('--dist-backend', default='nccl', type=str, help='')
parser.add_argument('--rank', default=0, type=int, help='')
parser.add_argument('--world_size', default=1, type=int, help='')
parser.add_argument('--distributed', action='store_true', help='')
args = parser.parse_args()

gpu_devices = ','.join([str(id) for id in args.gpu_devices])
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_devices


model = torchvision.models.resnet18(weights = 'DEFAULT')
model.fc = nn.Linear(model.fc.in_features, 10)

# summary(model, (1, 3, 224, 224), device = "cpu")
# model.to(device) # move to gpu
print("--- Model Loaded --- ")


def main():
    args = parser.parse_args()

    ngpus_per_node = torch.cuda.device_count()

    args.world_size = ngpus_per_node * args.world_size
    mp.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args))
        
        
def main_worker(gpu, ngpus_per_node, args):
    args.gpu = gpu
    ngpus_per_node = torch.cuda.device_count()    
    print("Use GPU: {} for training".format(args.gpu))
        
    args.rank = args.rank * ngpus_per_node + gpu    
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,
                            world_size=args.world_size, rank=args.rank)

    print('==> Making model..')
    net = model
    torch.cuda.set_device(args.gpu)
    net.cuda(args.gpu)
    args.batch_size = int(args.batch_size / ngpus_per_node)
    args.num_workers = int(args.num_workers / ngpus_per_node)
    net = torch.nn.parallel.DistributedDataParallel(net, device_ids=[args.gpu])
    num_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
    print('The number of parameters of model is', num_params)

    print('==> Preparing data..')
    transforms_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

    dataset_train = CIFAR10(root='./data', train=True, download=True, 
                            transform=transforms_train)
    train_sampler = torch.utils.data.distributed.DistributedSampler(dataset_train)
    train_loader = DataLoader(dataset_train, batch_size=args.batch_size, 
                              shuffle=(train_sampler is None), num_workers=args.num_workers, 
                              sampler=train_sampler)

    # there are 10 classes so the dataset name is cifar-10
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 
               'dog', 'frog', 'horse', 'ship', 'truck')
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=args.lr, 
                          momentum=0.9, weight_decay=1e-4)
    num_epochs = args.epochs
    for epoch in range(num_epochs):
        train(net, criterion, optimizer, train_loader, args.gpu)
            

def train(net, criterion, optimizer, train_loader, device):
    net.train()

    train_loss = 0
    correct = 0
    total = 0
    
    epoch_start = time.time()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        start = time.time()
        
        inputs = inputs.cuda(device)
        targets = targets.cuda(device)
        outputs = net(inputs)
        loss = criterion(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        acc = 100 * correct / total
        
        batch_time = time.time() - start
        
        if batch_idx % 20 == 0:
            print('Epoch: [{}/{}]| loss: {:.3f} | acc: {:.3f} | batch time: {:.3f}s '.format(
                batch_idx, len(train_loader), train_loss/(batch_idx+1), acc, batch_time))
    
    elapse_time = time.time() - epoch_start
    elapse_time = datetime.timedelta(seconds=elapse_time)
    print("Training time per epoch {}".format(elapse_time))
    

if __name__=='__main__':
    main()

Overwriting train.py


In [2]:
# cd dist_parallel
# For Kaggle T4x2 So we pass two ids
!python train.py --gpu_device 0 1  --batch_size 768 --epochs 5

--- Model Loaded --- 
--- Model Loaded --- 
Use GPU: 1 for training
--- Model Loaded --- 
Use GPU: 0 for training
==> Making model..
==> Making model..
The number of parameters of model is The number of parameters of model is11181642 
11181642==> Preparing data..

==> Preparing data..
Files already downloaded and verifiedFiles already downloaded and verified

--- Model Loaded --- 
--- Model Loaded --- 
--- Model Loaded --- 
--- Model Loaded --- 
Epoch: [0/66]| loss: 2.560 | acc: 11.458 | batch time: 0.807s 
Epoch: [0/66]| loss: 2.609 | acc: 13.802 | batch time: 0.931s 
Epoch: [20/66]| loss: 2.093 | acc: 34.772 | batch time: 0.095s 
Epoch: [20/66]| loss: 2.130 | acc: 34.338 | batch time: 0.092s 
Epoch: [40/66]| loss: 1.876 | acc: 42.124 | batch time: 0.096s Epoch: [40/66]| loss: 1.819 | acc: 42.537 | batch time: 0.093s 

Epoch: [60/66]| loss: 1.857 | acc: 43.327 | batch time: 0.097s 
Epoch: [60/66]| loss: 1.814 | acc: 43.716 | batch time: 0.096s 
Training time per epoch 0:00:32.633945
T